In [32]:
import pygame
from sys import exit
import sympy as sy
from sympy import Integer, simplify
from sympy import factorial as fact
import json

#from tkinter import *
#from tkinter import messagebox

def listr(x):  # Turns list to string
    x = [str(i) for i in x]
    return ' , '.join(x)

def int_term(seq):   # turn float type integer terms  into integer type in a sequence
    res = []
    for i in seq:
        if type(i)==float:
            if int(i)==i:
                res.append(int(i))
            else:
                res.append(i)
        else:
            res.append(i)
    return res

def check_operator(sli):
    global user_text
    res = list(map(sli.count, ['+','-','*','/','t']))
    return res

def insert_times(user_text):
    res = ''
    for i, n in enumerate(user_text[:-1]):
        if user_text[i:i+2][0] == '(' or user_text[i:i+2][1] == ')':    # '(S'  'S)'
            res += user_text[i]
        elif user_text[i:i+2].count('n') and check_operator(user_text[i:i+2]) == [0,0,0,0,0]:
            res += user_text[i] + '*'
        elif user_text[i:i+2][1] in ('(' , 'f')   and check_operator(user_text[i:i+2]) == [0,0,0,0,0]:
            res += user_text[i] + '*'
        elif user_text[i:i+2][0] == ')' and check_operator(user_text[i:i+2]) == [0,0,0,0,0]:
            res += user_text[i] + '*'
        else:
            res += user_text[i]
    res += user_text[-1]
    return res

def syint(expr):         # capture an integer and turn it into sympy Integer
    num_list = [str(i) for i in range(10)]
    num = ''
    res = ''
    if expr[0] in num_list:
        res += 'Integer('
    for i, n in enumerate(expr[:-1]):
        if expr[i:i+2][1] in num_list and expr[i:i+2][0] not in num_list:
            res += f'{n}Integer('
        elif expr[i:i+2][0] in num_list and expr[i:i+2][1] not in num_list:
            res += f'{n})'
        else:
            res += n
    res += expr[-1]
    if expr[-1] in num_list:
        res += ')'
    return res


pygame.init()

res = (720,720)
screen = pygame.display.set_mode(res)
pygame.display.set_caption('Make The Sequence')
clock = pygame.time.Clock()

color = (255,255,255)
# light shade of the button
color_light = (130,130,130)
  
# dark shade of the button
color_dark = (100,100,100)
color_darker = (60,60,60)

color_green = (63,105,105)
color_gold = (224,169,57)
color_gold_2 = (214,159,47)
color_red = (255,120,120)

test_font = pygame.font.SysFont('Cambria Math', 50)
math_font = pygame.font.SysFont('Cambria Math', 50, italic=False)
small_math_font = pygame.font.SysFont('Cambria Math', 30, italic=False)
smaller_math_font = pygame.font.SysFont('Cambria Math', 25, italic=False)
custom_math_font = pygame.font.SysFont('Cambria Math', 20, italic=False)

ori_font = pygame.font.SysFont(None, 35)

smallfont = pygame.font.SysFont('Corbel',35)
titlefont = pygame.font.SysFont('Cambria Math', 70, italic=False)
text = smallfont.render('quit' , True , color)
title = titlefont.render('Make  The  Sequence' , True , color)

width = screen.get_width()
  
# stores the height of the
# screen into a variable
height = screen.get_height()

checked = 0
passed = 0
win = 0

dummy = False    # disable buttons
error_text = small_math_font.render('Invalid input', True, color_red)
show_error_text = False

prev_state = 'title_screen'

#unbeaten_levels = [i for i in range(1,61)]
#unbeaten_levels_text = small_math_font.render(listr(unbeaten_levels), True, color)

empty_save_data = []
save_data = []

try:
    with open('MTS_completed_levels_list.txt') as level_file:
        save_data = json.load(level_file)
except Exception:
    with open('MTS_completed_levels_list.txt','w') as level_file:
        json.dump(empty_save_data, level_file)
    with open('MTS_completed_levels_list.txt') as level_file:
        save_data = json.load(level_file)
completed_levels = set(save_data)

incompleted_levels = set(list(range(1,61))) - completed_levels


class Button():           # clickable buttons
    def __init__(self,x,y, text=None, image=None, scale=None):
        if image:  # if image argument is inputed
            width = image.get_width()
            height = image.get_height()
            self.image = image
            self.rect = self.image.get_rect()
            self.pos = (x,y)
            self.rect.topleft = self.pos
            if scale:
                self.image = pygame.transform.scale(image,(int(width * scale), int(height * scale )))
                
        else:  # no importing image
            self.image = None
            self.pos = (x,y)
        
        if text:
            self.text = smallfont.render(text , True , color)
        self.clicked = False
        
            
    def draw(self,width,height, text_align_x=3, text_align_y=5, button_color=color_dark): 
        global dummy
        # draw button on screen
        action = False
        
        if self.image:
            screen.blit(self.image, (self.rect.x, self.rect.y))
        else:
            self.rect = pygame.Rect(self.pos[0],self.pos[1] , width, height )
            pygame.draw.rect(screen,button_color,[self.pos[0],self.pos[1] ,width,height])
            screen.blit(self.text , (self.pos[0] + width/text_align_x, self.pos[1] + height/text_align_y))
        
        if dummy: return   # non-clickable buttons
    
        mouse_pos = pygame.mouse.get_pos()
        if self.rect.collidepoint(mouse_pos):
            if not self.image:
                pygame.draw.rect(screen,color_light,[self.pos[0],self.pos[1] ,width,height])
                screen.blit(self.text , (self.pos[0] + width/text_align_x, self.pos[1] + height/text_align_y))
            if pygame.mouse.get_pressed()[0] and self.clicked == False:
                self.clicked = True
                action = True
            
        if pygame.mouse.get_pressed()[0] == 0: 
            self.clicked = False
              
        return action
    
    def draw_immune(self,width,height, text_align_x=3, text_align_y=5, button_color=color_dark): 
        # immune to dummy spell, so dummy == True  won't disable this button
        # draw button on screen
        action = False
        
        if self.image:
            screen.blit(self.image, (self.rect.x, self.rect.y))
        else:
            self.rect = pygame.Rect(self.pos[0],self.pos[1] , width, height )
            pygame.draw.rect(screen,button_color,[self.pos[0],self.pos[1] ,width,height])
            screen.blit(self.text , (self.pos[0] + width/text_align_x, self.pos[1] + height/text_align_y))
    
        mouse_pos = pygame.mouse.get_pos()
        if self.rect.collidepoint(mouse_pos):
            if not self.image:
                pygame.draw.rect(screen,color_light,[self.pos[0],self.pos[1] ,width,height])
                screen.blit(self.text , (self.pos[0] + width/text_align_x, self.pos[1] + height/text_align_y))
            if pygame.mouse.get_pressed()[0] and self.clicked == False:
                self.clicked = True
                action = True
            
        if pygame.mouse.get_pressed()[0] == 0: 
            self.clicked = False
               
        return action
    
    def draw_release(self,width,height, text_align_x=3, text_align_y=5, button_color=color_dark): 
        # draw button on screen that executes when mouse button is released
        action = False
        
        if self.image:
            screen.blit(self.image, (self.rect.x, self.rect.y))
        else:
            self.rect = pygame.Rect(self.pos[0],self.pos[1] , width, height )
            pygame.draw.rect(screen,button_color,[self.pos[0],self.pos[1] ,width,height])
            screen.blit(self.text , (self.pos[0] + width/text_align_x, self.pos[1] + height/text_align_y))
        
    
        mouse_pos = pygame.mouse.get_pos()
        if self.rect.collidepoint(mouse_pos):
            if not self.image:
                pygame.draw.rect(screen,color_light,[self.pos[0],self.pos[1] ,width,height])
                screen.blit(self.text , (self.pos[0] + width/text_align_x, self.pos[1] + height/text_align_y))
                
            for event in pygame.event.get():
                if event.type == pygame.MOUSEBUTTONUP:
                    self.clicked = True
                    action = True
                else:
                    self.clicked = False
            
            
        
        return action
    

# title screen buttons

start_button = Button(270,250,'Start')
level_select_button = Button(270,350,'Level  Select')
instructions_button = Button(270,450,'Instructions')
exit_button = Button(270,550,'Exit')



back_button = Button(260,620,'Back to menu')
back_button_2 = Button(315,20,'BACK')
    

class Text():
    def __init__(self, x, y, text, color='White', size=50, title=False):
        
        if title:
            self.font = pygame.font.SysFont('Cambria Math', size)
            self.text = self.font.render(text , True , color)
        else:
            self.font = pygame.font.SysFont('Corbel', size)
            self.text = self.font.render(text , True , color)
        self.size = size
        self.pos = (x,y)
        
        screen.blit(self.text , (self.pos[0] , self.pos[1] ))


show_message = True

class Message():
    def __init__(self, x, y, *text , x_offset=20):
        global show_message, dummy
        self.text = text
        self.display = True
        self.button = Button(305,400, 'OK')
        self.pos = (x,y)
        
        if show_message:
            dummy = True
            darken_bg = pygame.Surface((1000,750))  # the size of your rect
            darken_bg.set_alpha(128)                # alpha level
            darken_bg.fill((50,50,50))           # this fills the entire surface
            screen.blit(darken_bg, (0,0))
            pygame.draw.rect(screen, color_darker, [x,y , 320, 250])
            for i, n in enumerate(text):
                message_text = Text(x+x_offset, y+20+i*40, n, 'White', 30)
            if self.button.draw_immune(140,70, 3, 5, color_dark):
                show_message = False
                dummy = False
                
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                if event.type == pygame.KEYDOWN:
                    if event.key in (pygame.K_SPACE, pygame.K_ESCAPE):
                        show_message = False
                        dummy = False
        else:
            pass

# levels

next_level_button = Button(500,70,'Next level')

def get_level_text(n):
    global level_text, cur_level
    if cur_level < last_level:
        level_text = test_font.render(f'Level {n}' , True , 'Yellow')
    else:
        if completed_levels == set([i for i in range(1,61)]):
            level_text = test_font.render('Congrats! You beat the game!' , True , 'Yellow') 
        else:
            level_text = test_font.render('Please complete all the levels.' , True , 'Yellow') 
    return f'Level {n}'

cur_level = 0
increased_char_limit_level = 45
increased_char_limit_level_2 = 53
long_levels = [53,58,59,60]    # level with super long sequence

# return to save_level after going to menu screen and press start again
save_level = 1
if incompleted_levels:
    save_level = min(incompleted_levels)
else:
    save_level = 60

def start_level():
    global title_screen, run_levels, cur_level, prev_state
    cur_level = save_level      # click start button --> jump to level  <cur_level>
    title_screen = False 
    run_levels = True
    prev_state = 'title_screen'

def level(n):
    global cur_level, level_text, win, passed, char_limit
    cur_level = n
    get_level_text(n)
    
    if n < last_level:
        screen.blit(level_text , (290, 70))
        if n in [14,17,15,16,22,26,27,28,29,30,39,40,41,42,43,50,52,56,57]:               # levels with  small sequence text 
            screen.blit(seq_text(seq_list[cur_level], True) , (200, 160))
        elif n in long_levels:    # tiny sequence text
            screen.blit(seq_text(seq_list[cur_level], 20) , (150, 170))

        else:                                # big sequence text
            screen.blit(seq_text(seq_list[cur_level],) , (200, 150))
            
    else:     # last level display text
        screen.blit(level_text , (55, 70))
    
    if n < increased_char_limit_level:     # custom character limit
        char_limit = 15
    
    if n >= increased_char_limit_level:     # custom character limit
        char_limit = 20
    if n in long_levels:     # custom character limit
        char_limit = 50
    if n == 54:
        char_limit = 40
    if n in (55,57):
        char_limit = 15
    if n == 56:
        char_limit = 20
    if n == 58:
        char_limit = 40
    if n == 59:
        char_limit = 32
    if n == 60:
        char_limit = 25
        
seq_1 = [1,2,3,4,5]

seq_2 = [2,4,6,8,10]

seq_list = ['title screen', 
           [1,2,3,4,5], 
           [2,4,6,8,10],
           [4,5,6,7,8],
           [1,1,1,1,1],
           [1,3,5,7,9],
           [5,9,13,17,21],
           [13,23,33,43,53],
           [38,31,24,17,10],
           [1,4,9,16,25],
           [1,3,6,10,15],
            [1,8,27,64,125],
            [1,2,4,8,16],
            [1,2,4,7,11],
            [3,9,27,81,243],
            [48,72,108,162,243],
            [1,11,111,1111,11111],
            [3,-6,9,-12,15],    
            [5,13,25,41,61],
            [1,5,12,22,35],
            [1,5,14,30,55],
             [1,0,1,0,1],
             [23,45,89,177,353],
             [1,4,9,18,35],
             [1,9,36,100,225],
             [17,69,17,69,17],
             [4,16,64,256,1024],
             [2,16,512,65536,33554432],
             [1,4,27,256,3125],
             [2,4,16,256,65536],
             [1,12,108,864,6480],
              [1,1,2,2,3],
              [1,3,4,6,7],
              [1,2,3,4,4],
              [1,2,0,1,2],
              [1,2,3,2,1],
              [1,0,0,0,0],
              [1,2,3,5,9],
              [6,14,36,98,276],
              [5,32,176,896,4352],
              [-1,6,49,244,1023],
               [1701,7168,21875,54432,117649],
               [1,121,12321,1234321,123454321],
               [648,2160,5688,13920,33128],
               [1,2,3,5,8],
               [50,67,75,80,83],
               [2,3,5,7,11],
               [4,7,15,29,59],
               [20,24,29,33,38],
               [4,2,6,4,8],
               [62,363,1364,3905,9330],
                [7,5,8,4,9],
                [5,8,13,21,34],
                [1,1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610],
                [2,6,24,120,720],
                [3,6,18,72,360],
                [36,54,108,270,810],
                [7,28,252,4032,100800],
                [23,46,138,276,828,1656,4968,9936,29808,59616],
                [1, 19, 171, 969, 3876, 11628, 27132, 50388, 75582, 92378],
                [0,1,2,9,44,265, 1854, 14833, 133496, 1334961],
               'last level'        # beat the game
              ]

last_level = len(seq_list) - 1

# Level Interface

def seq_text(seq, small=0):    # make the sequence text displayed on screen
    seq = listr(seq)
    if small == True:
        return small_math_font.render(seq + ' ,  . . . ',True , color)
    elif type(small)==int and small > 0:
        custom_math_font = pygame.font.SysFont('Cambria Math', small, italic=False)
        return custom_math_font.render(seq + ' ,  . . . ',True , color)
    else:
        return math_font.render(seq + ' ,  . . . ',True , color)

seq1_text = math_font.render('1 , 2 , 3 , 4 , 5 , . . . ',True , color)
seq2_text = math_font.render('2 , 4 , 6 , 8 , 10 , . . . ',True , color)
a_n = math_font.render('a(n)=',True , color)
yours_text = small_math_font.render('Input: ',True , color)
target_text = small_math_font.render('Target: ',True , color)

input_level =  math_font.render('Level=               (From 1 to 60)',True , color)

char_count = 0       # character limit
char_limit = 15
char_limit_color = 'White'
char_limit_count = small_math_font.render(f'{char_count}/15',True , char_limit_color)  # display char_count
char_limit_text = small_math_font.render('Character used',True , color)


level_complete_text = small_math_font.render('Level complete!',True , color)
passed_indicator =  math_font.render('O',True , 'Green')    #'O X indicator'
failed_indicator =  math_font.render('X',True , (255,100,100))



# player input using keyboard
user_text = '_'
input_rect = pygame.Rect(150,300, 450, 70)
input_color = pygame.Color('lightskyblue3')

char_allowed = [str(i) for i in range(10)] + ['+' , '-', '*', '/', '(', ')', 'n']
num_list = [str(i) for i in range(10)]


# keypad button functions

def delete():
    global char_count, user_text, char_limit_color
    if len(user_text) > 1:
        char_limit_color = 'White'
        for i, n in enumerate(user_text):
            if n == '_' and i != 0:
                if 'fact(' in user_text:
                    if user_text[i-5:i] == 'fact(':
                        user_text = user_text[:i-5] + '_' + user_text[i+1:]
                        char_count -= 5
                    else:
                        user_text = user_text[:i-1] + '_' + user_text[i+1:]
                        char_count -= 1
                else:
                    user_text = user_text[:i-1] + '_' + user_text[i+1:]
                    char_count -= 1

            
def all_clear():
    global checked, passed, char_count, user_text, char_limit_color
    char_limit_color = 'White'
    user_text = '_'
    char_count = 0
    checked = 0
    passed = 0
            
def append(char):
    global char_count, user_text, char_limit_color
    if char_count < char_limit:
        user_text = user_text.replace('_', char + '_')
        char_count += 1
    if char_count == char_limit :
        
        char_limit_color = (255,120,120)
    else:
        pass

    
def true_user_text(user_text):   # get rid of the underscore at the end of user_text
    res = ''
    for i in user_text:
        if i != '_':
            res += i
    return res
    
def check():
    global seq_list, user_text, checked, win, passed, user_seq_text, level_complete_text, cur_level, completed_levels
    checked = 0
    passed = 0
    user_seq_str = ''
    user_seq_text = ''
    num_of_terms = len(seq_list[cur_level])
    if user_text == '_': 
        user_seq_text = small_math_font.render('Invalid input',True , color)
        checked = 1
        return
    mod_user_text = true_user_text(user_text)
    mod_user_text = insert_times(mod_user_text)   # make a python-readable user_text
    mod_user_text = syint(mod_user_text)         # turn integer into sympy Integer
    if cur_level <= 30 or cur_level in (53,54):    # disable floor division
        mod_user_text = mod_user_text.replace('//','??')

    try:
        user_seq = [eval(mod_user_text) for n in range(1,num_of_terms+1)]
        
    except ZeroDivisionError:
        user_seq_text = small_math_font.render('Math error',True , color)
        checked = 1
    except Exception: 
        user_seq_text = small_math_font.render('Invalid input',True , color)
        checked = 1
    
    else:
        user_seq = [simplify(i) for i in user_seq]
        user_seq_str = listr(user_seq)

        if cur_level in long_levels:
            user_seq_text = custom_math_font.render(user_seq_str,True , color)
        else:
            user_seq_text = small_math_font.render(user_seq_str,True , color)
    
        checked = 1

        if user_seq == seq_list[cur_level]:
            passed = 1
            win = 1
            completed_levels.add(cur_level)
            
def left():    # make the underscore move left by pressing <-- (left)
    global char_count, user_text, char_limit_color
    for i, n in enumerate(user_text):
        if n == '_' and i != 0:
            if 'fact(' in user_text:
                if user_text[i-5:i] == 'fact(':
                    user_text = user_text[:i-5] + '_' + user_text[i-5:i] + user_text[i+1:]
                else:
                    user_text = user_text[:i-1] + '_' + user_text[i-1] + user_text[i+1:]
            else:
                user_text = user_text[:i-1] + '_' + user_text[i-1] + user_text[i+1:]
            
def right():    # make the underscore move right by pressing --> (right)
    global char_count, user_text, char_limit_color
    for i, n in enumerate(user_text):
        if n == '_' and i != char_count :
            if 'fact(' in user_text:
                if user_text[i+1:i+6] == 'fact(':
                    user_text = user_text[:i] + user_text[i+1:i+6] + '_' + user_text[i+6:]
                else:
                    user_text = user_text[:i] + user_text[i+1] + '_'  + user_text[i+2:]
            else:
                user_text = user_text[:i] + user_text[i+1] + '_'  + user_text[i+2:]
                
                
def leftmost():
    global char_count, user_text, char_limit_color
    user_text = '_' + user_text.replace('_','')
    
def rightmost():
    global char_count, user_text, char_limit_color
    user_text =  user_text.replace('_','') + '_'
    
    
def user_level_convert(user_level):
    if user_level == '_':
        return ''
    return int(user_level[:-1])

def go_to_level(user_level):
    global cur_level, run_levels, level_select_page, show_error_text, prev_state
    show_error_text = False
    if user_level == '':
        show_error_text = True
        return
    
    if 1 <= user_level <= 60:
        level_select_page = False
        cur_level = user_level
        all_clear()
        run_levels = True
        show_error_text = False
        prev_state = 'level_select_page'
    else:
        show_error_text = True
        

def return_to_menu():
    global save_level, cur_level, win, run_levels, user_text, prev_state, title_screen, level_select_page
    save_level = cur_level
    cur_level = 0
    win = 0
    run_levels = False
    user_text = '_'
    if prev_state == 'title_screen':
        title_screen = True
    elif prev_state == 'level_select_page':
        level_select_page = True
    else:
        title_screen = True
        
            
# keypad 

key_0 = Button(100,555,'0')
key_1 = Button(175,555,'1')
key_2 = Button(250,555,'2')
key_3 = Button(325,555,'3')
key_4 = Button(400,555,'4')
key_5 = Button(100,480,'5')
key_6 = Button(175,480,'6')
key_7 = Button(250,480,'7')
key_8 = Button(325,480,'8')
key_9 = Button(400,480,'9')

key_plus = Button(495,555,'+')
key_minus = Button(570,555,'-')
key_times = Button(495,480,'*')
key_divides = Button(570,480,'/')
key_power = Button(250,400,'^')
key_floor_div = Button(175,400,'/ /')

key_n = Button(250,630,'n')
key_left_bracket = Button(325,400,'(')
key_right_bracket = Button(400,400,')')

key_left = Button(175,630,'<')
key_right = Button(325,630,'>')
key_leftmost = Button(100,630,'<<')
key_rightmost = Button(400,630,'>>')

key_fact = Button(100,400,'n!')

key_del = Button(495,400,'DEL')
key_ac = Button(570,400,'AC')

key_check = Button(495,630,'CHECK')


# level select keypad

key_l0 = Button(100,530,'0')
key_l1 = Button(175,530,'1')
key_l2 = Button(250,530,'2')
key_l3 = Button(325,530,'3')
key_l4 = Button(400,530,'4')
key_l5 = Button(100,455,'5')
key_l6 = Button(175,455,'6')
key_l7 = Button(250,455,'7')
key_l8 = Button(325,455,'8')
key_l9 = Button(400,455,'9')

key_ldel = Button(495,455,'DEL')
key_lac = Button(570,455,'AC')
key_lenter = Button(495,530,'ENTER')



# go_to_level buttons

level_button_size = 60

next_page_button = Button(600,30, 'NEXT')
prev_page_button = Button(30,30, 'PREV')

level_button_list = [0]
level_button_list += [Button( 40 + (level_button_size+5) * ((i-1)%10) ,120+((i-1)//10)*65, str(i) ) for i in range(1,31)]
level_button_list2 = [0]
level_button_list2+=[Button( 40 + (level_button_size+5) * ((i-1)%10) ,120+((i-31)//10)*65, str(i) ) for i in range(31,61)]


page = 0

def level_button_next_page():
    global page, level_button_list, level_button_list2, completed_levels
    if page == 0: 
        page = 1

    elif page == 1: 
        page = 0

    

    
### pages control ###

title_screen = True
run_levels = False

# Level select page

level_select_page = False


# Instructions page
instructions_page = False
description_str1 = 'Your goal is to make the sequence given in each level' 
description_str2 = 'by typing an appropriate formula in the box using'
description_str3 = 'a limited number of characters.'
description_str4 = 'You can use the various buttons shown below to'
description_str5 = 'type the formula.'

# game loop

run = True
while run:
    
    
    for ev in pygame.event.get():
          
        if ev.type == pygame.QUIT:
            # save list of completed levels
            save_data = list(completed_levels)
            with open('MTS_completed_levels_list.txt','w') as level_file:
                json.dump(save_data, level_file)
            run = False
            
        if title_screen:
            all_clear()
            if ev.type == pygame.KEYDOWN:
                if ev.key == pygame.K_ESCAPE:
                    
                    # save list of completed levels
                    save_data = list(completed_levels)
                    with open('MTS_completed_levels_list.txt','w') as level_file:
                        json.dump(save_data, level_file)
                    run = False
                    
                elif ev.key == pygame.K_SPACE:
                    start_level()
                elif ev.unicode in ('H','h'):
                    title_screen = False
                    instructions_page = True
                    
                elif ev.unicode in ('S','s'):
                    title_screen = False
                    level_select_page = True
                    
        if not title_screen and not run_levels:
            if ev.type == pygame.KEYDOWN:
                if ev.key == pygame.K_ESCAPE:
                    instructions_page = False
                    level_select_page = False
                    title_screen = True
       
        if (not title_screen) and cur_level:
            if ev.type == pygame.KEYDOWN:
                if ev.key == pygame.K_BACKSPACE:
                    delete()
                elif ev.unicode in char_allowed:
                    append(ev.unicode)
                elif ev.unicode in ('r','R'):
                    all_clear()
                elif ev.unicode == '^':
                    append('*')
                    append('*')
                elif  ev.unicode == '!' and char_count < char_limit - 4:
                    append('f')
                    append('a')
                    append('c')
                    append('t')
                    append('(')
                elif ev.key == pygame.K_RETURN or ev.key == pygame.K_KP_ENTER:
                    check()
                elif ev.key == pygame.K_LEFT:
                    left()
                elif ev.key == pygame.K_RIGHT:
                    right()
                elif ev.unicode in (',','<'):
                    leftmost()
                elif ev.unicode in ('.','>'):
                    rightmost()
                
                elif ev.key == pygame.K_SPACE and win == 1:
                    cur_level += 1
                    all_clear()
                    win = 0
                    passed = 0
                    checked = 0
                    level(cur_level)
                
                elif ev.key == pygame.K_ESCAPE:     # Press ESC in a level
                    return_to_menu()
                
                else:
                    pass
        
        if level_select_page:
            if ev.type == pygame.KEYDOWN:
                if ev.key == pygame.K_BACKSPACE:
                    delete()
                elif ev.unicode in num_list:
                    append(ev.unicode)
                elif ev.unicode in ('r','R'):
                    all_clear()
                elif ev.key == pygame.K_RETURN or ev.key == pygame.K_KP_ENTER:
                    go_to_level(user_level_convert(user_text))
        
        
                  
    screen.fill((94,129,162))
    
    if title_screen:       # display title screen
        screen.blit(title , (60, 70))
        show_error_text = False
        if start_button.draw(200,60):
            start_level()
            
        if level_select_button.draw(200,60,13):
            title_screen = False
            level_select_page = True
            
            
        if exit_button.draw(200,60,2.7):
            run = False
        
        if instructions_button.draw(200,60,10):
            title_screen = False
            instructions_page = True
            
    # Level Select page
    if level_select_page:
        level_select_title = Text(170,40, 'Type / Select Level', 'Yellow', 50, True)
        
        char_limit = 2
        
        if key_l0.draw(70,70): append('0')
        if key_l1.draw(70,70):    append('1')
        if key_l2.draw(70,70):    append('2')
        if key_l3.draw(70,70):    append('3')
        if key_l4.draw(70,70):    append('4')
        if key_l5.draw(70,70):    append('5')
        if key_l6.draw(70,70):    append('6')
        if key_l7.draw(70,70):    append('7')
        if key_l8.draw(70,70):    append('8')
        if key_l9.draw(70,70):    append('9')
            
        if key_ldel.draw(70,70,9): delete()
        if key_lac.draw(70,70,5):  all_clear()
        if key_lenter.draw(145,70,7):
            go_to_level(user_level_convert(user_text))
        
        
        screen.blit(input_level , (20, 360))
        text_surf = math_font.render(user_text, True, color)
        input_rect = pygame.Rect(180,360, 90, 70)
        screen.blit(text_surf, (190,360))       # user inputted text       
        pygame.draw.rect(screen, input_color, input_rect, 2)   # text box
        
        #screen.blit(unbeaten_levels_text, (20,120)) 
        if page == 0:
            for i,n in enumerate(level_button_list):
                if i == 0: continue
                if i not in completed_levels:
                    if n.draw(level_button_size,level_button_size, 4, 5, color_green):
                        go_to_level(i)
                else:
                    if n.draw(level_button_size,level_button_size, 4, 5, color_gold_2):
                        go_to_level(i)
        else:
            for i,n in enumerate(level_button_list2):
                if i == 0: continue
                if i+30 not in completed_levels:
                    if n.draw(level_button_size,level_button_size, 4, 5, color_green):
                        go_to_level(i+30)
                else:
                    if n.draw(level_button_size,level_button_size, 4, 5, color_gold_2):
                        go_to_level(i+30)
                
        if next_page_button.draw(100,70,9,4):
            level_button_next_page()
        if prev_page_button.draw(100,70,9,4):
            level_button_next_page()
        
        if show_error_text:
            screen.blit(error_text, (10,410))
        
        if back_button.draw(220,70,13,4):
            level_select_page = False
            title_screen = True
    
    # Instructions page
    if instructions_page:
        instructions_title = Text(240,40, 'Instructions', 'Yellow', 50, True)
        descriptions_l1 = Text(40,120, description_str1, 'White', 30)
        descriptions_l2 = Text(40,160, description_str2, 'White', 30)
        descriptions_l3 = Text(40,200, description_str3, 'White', 30)
        descriptions_l4 = Text(40,240, description_str4, 'White', 30)
        descriptions_l5 = Text(40,280, description_str5, 'White', 30)
        
        dumkey_plus = Button(40,340,'+')           # buttons showcase
        dumkey_plus.draw(50,50,3,7,color_dark)
        dumkey_minus = Button(40,400,'-')          
        dumkey_minus.draw(50,50,2.5,7,color_dark)
        dumkey_times = Button(40,460,'*')          
        dumkey_times.draw(50,50,3,5,color_dark)
        dumkey_divides = Button(40,520,'/')          
        dumkey_divides.draw(50,50,2.5,7,color_dark)
        
        des_plus = Text(110,350, 'Plus', 'White', 30)
        des_minus = Text(110,410, 'Minus', 'White', 30)
        des_times = Text(110,470, 'Times', 'White', 30)
        des_divides = Text(110,530, 'Divides', 'White', 30)
        
        dumkey_left_bracket = Button(250,340,'(')           
        dumkey_left_bracket.draw(50,50,3,7,color_dark)
        dumkey_right_bracket = Button(250,400,')')           
        dumkey_right_bracket.draw(50,50,2.5,7,color_dark)
        dumkey_power = Button(250,460,'^')           
        dumkey_power.draw(50,50,3,5,color_dark)
        dumkey_floor_div = Button(250,520,'/ /')           
        dumkey_floor_div.draw(50,50,3.75,7,color_dark,)
        
        des_left_bracket = Text(320,350, 'Left bracket', 'White', 30)
        des_right_bracket = Text(320,410, 'Right bracket', 'White', 30)
        des_power = Text(320,470, 'Power', 'White', 30)
        des_floor_div = Text(320,530, 'Floor division', 'White', 30)
        
        dumkey_n = Button(520,340,'n')
        dumkey_n.draw(50,50)
        des_n = Text(590,350, 'Variable', 'White', 30)
        
        if back_button.draw(220,70,13,4):
            instructions_page = False
            title_screen = True
    
    # run levels
    if run_levels:
        level(cur_level)
        # draw clickable keypad on screen
        if key_0.draw(70,70): append('0')
        if key_1.draw(70,70):    append('1')
        if key_2.draw(70,70):    append('2')
        if key_3.draw(70,70):    append('3')
        if key_4.draw(70,70):    append('4')
        if key_5.draw(70,70):    append('5')
        if key_6.draw(70,70):    append('6')
        if key_7.draw(70,70):    append('7')
        if key_8.draw(70,70):    append('8')
        if key_9.draw(70,70):    append('9')
        
        if key_plus.draw(70,70):   append('+')
        if key_minus.draw(70,70):  append('-')
        if key_times.draw(70,70):  append('*')
        if key_divides.draw(70,70):append('/')
        if key_power.draw(70,70):  
            append('*')
            append('*')
        if cur_level > 30 and cur_level not in [53, 54]:   # floor divison banned in level 53,54
            if key_floor_div.draw(70,70):
                append('/')
                append('/')
                
        if cur_level >= 55:    # factorial button
            if key_fact.draw(70,70) and char_count < char_limit - 4:
                append('f')
                append('a')
                append('c')
                append('t')
                append('(')
        
        if key_n.draw(70,70):      append('n')
        if key_left_bracket.draw(70,70): append('(')
        if key_right_bracket.draw(70,70):append(')')
            
        if key_left.draw(70,70): left()
        if key_right.draw(70,70): right()
        if key_leftmost.draw(70,70): leftmost()
        if key_rightmost.draw(70,70): rightmost()
        
        if key_del.draw(70,70, 9): delete()
        if key_ac.draw(70,70, 5):
            all_clear()
                
        if key_check.draw(145,70,7): check()       
        
        if back_button_2.draw(100,50,10): return_to_menu()
        
        screen.blit(a_n , (20, 300))
        screen.blit(target_text , (50, 160))
        screen.blit(yours_text , (55, 220))
        
        
        if char_limit <= 15:
            text_surf = math_font.render(user_text, True, color)
            input_rect = pygame.Rect(150,300, 490, 70)
            screen.blit(text_surf, (160,300))       # user inputted text       
            pygame.draw.rect(screen, input_color, input_rect, 2)   # text box
        else:
            if cur_level in [53,54,58]:   # very small math font
                text_surf = custom_math_font.render(user_text, True, color)
                input_rect = pygame.Rect(140,300, 570, 70)
                screen.blit(text_surf, (145,320))       # user inputted text       
                pygame.draw.rect(screen, input_color, input_rect, 2)   # text box
            
            else:   # smaller math font
                text_surf = small_math_font.render(user_text, True, color)
                input_rect = pygame.Rect(150,300, 490, 70)
                screen.blit(text_surf, (160,315))       # user inputted text       
                pygame.draw.rect(screen, input_color, input_rect, 2)   # text box
        
        if cur_level < last_level:
            char_limit_count = small_math_font.render(f'{char_count}/{char_limit}',True , char_limit_color)
            screen.blit(char_limit_count, (100,70))   # character limit indicator
            screen.blit(char_limit_text, (55,20))
        
        if checked == 1:                 # display user sequence and pass/fail indicator
            if cur_level in long_levels:
                screen.blit(user_seq_text, (150,230))
            else:
                screen.blit(user_seq_text, (200,220))
            if passed == 1:
                screen.blit(passed_indicator, (650,300))
            else: 
                screen.blit(failed_indicator, (650,300))
                        
        if win == 1:       # level complete
            screen.blit(level_complete_text, (500,20))
            if next_level_button.draw(200,70,6,5, (224,169,57)):
                cur_level += 1
                all_clear()
                win = 0
                passed = 0
                checked = 0
                level(cur_level)
        
        if cur_level in [31]:
            message = Message(210,250,'   Floor division', 'button unlocked.', x_offset=60)
        if cur_level in [increased_char_limit_level]:
            message = Message(210,250,'The character limit has', 'been increased to 20.')
        if cur_level in [increased_char_limit_level_2]:
            message = Message(210,250,'The character limit may', 'vary from level to level','from now on.')
        if cur_level in [55]:
            message = Message(210,250,'Factorial button unlocked.', x_offset=5)
            
    pygame.display.update()

    

    
pygame.quit()

TypeError: Invalid comparison of non-real zoo